# Lesson 3: Enable Logging

### Import all needed packages

In [1]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [2]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [3]:
log_group_name = '/my/amazon/bedrock/logs'

In [4]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' created successfully.


In [5]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [6]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'ab31e96d-a4b3-4356-9274-179a8fedc76d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 15 Feb 2024 17:38:46 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ab31e96d-a4b3-4356-9274-179a8fedc76d'},
  'RetryAttempts': 0}}

In [7]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': 'c42881d6-c34e-4747-aff4-d9f736e4ed8d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 15 Feb 2024 17:38:54 GMT',
   'content-type': 'application/json',
   'content-length': '572',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c42881d6-c34e-4747-aff4-d9f736e4ed8d'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::810113758748:role/c99355a2566044l5845840t1w81011375874-LoggingIAMRole-kbrUpV6gCRxS',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l5845840t1w810113758-loggings3bucket-k9pbey26ficx',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l5845840t1w810113758-loggings3bucket-k9pbey26ficx',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [8]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [9]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet in the science fiction literature of author Douglas Adams. It is described as a planet similar to Earth in terms of size, mass, and atmospheric composition, but with a unique and strange landscape. The planet is home to a wide range of creatures, including sentient beings, animals, and plants.

One of the most notable features of Foobar is its "animal problem." The planet is overpopulated with animals, many of which are aggressive and dangerous. The animals have evolved to survive in a harsh environment, and they have developed a complex social structure that is often difficult for humans to understand.

Despite the challenges of living on Foobar, humans have managed to establish a small colony on the planet. The colony is governed by a group of scientists and engineers who are working to study the planet's animals and learn how to coexist with them. The colony is also home to a group of artists and musicians who are inspired by the planet's unique landsca

In [10]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-02-15T17:39:12Z",
    "accountId": "810113758748",
    "identity": {
        "arn": "arn:aws:sts::810113758748:assumed-role/voclabs/user3079678=eyJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcHAiLCJzdWIiOiIyMzQ"
    },
    "region": "us-west-2",
    "requestId": "7e5bd8f7-b670-4dd6-b67e-8bd60a97efa1",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJ

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [11]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [ ]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
